In [1]:
# add path to sispeuede to sys.path in python
import sys
import pathlib
import warnings
warnings.filterwarnings("ignore")


path_git = pathlib.Path("/Users/usuario/git")
for subdir in [
    "sisepuede", 
    "sisepuede_data_pipeline",
    # "sisepuede_juypyter"
]:
    path_cur = path_git.joinpath(subdir)
    if str(path_cur) not in sys.path:
        sys.path.append(str(path_cur))

path_pipeline = path_git.joinpath("sisepuede_data_pipeline")


import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import sisepuede.legacy.data_api as api
import sisepuede.manager.sisepuede_examples as sxl
import sisepuede.manager.sisepuede_file_structure as sfs
import sisepuede.manager.sisepuede_models as sm
import sisepuede.visualization.plots as spp
import sisepuede.utilities._plotting as spu
import sisepuede.core.support_classes as sc
import sisepuede.utilities._toolbox as sf
import time
from typing import *


import lib.process_utilities as pu
import lib.sisepuede_data_constructs as dc
import lib._util as lutil




In [2]:
##  Option to determine which columns are Peru specific - 20250807

In [ ]:
repo_old = api.SISEPUEDEBatchDataRepository(
    "/Users/usuario/git/sisepuede_data", 
    matt,
)

In [ ]:
df_old = repo_old.read(None)

In [ ]:
df_old_per = df_old[
    df_old["iso_code3"].isin(["PER"])
].drop(columns = ["iso_code3", "year"])

cols = [x for x in df_old_per.columns if df_old_per[x].dropna().shape[0] > 0]
cols_new = [x for x in df_from_pipeline.columns if x not in [regions.field_iso, time_periods.field_year]]
all_cols_peru = sorted(list(set(cols) | set(cols_new)))

# 
df_peru_fields_info = matt.build_variable_dataframe_by_sector(
    None,
    include_model_variable = True,
    include_model_variable_attributes = True,
    include_time_periods = False,
)
df_peru_fields_info["peru_specific_data_20250808"] = [
    1 if row["variable_field"] in all_cols_peru else 0 
    for i, row in df_peru_fields_info.iterrows()
]
df_peru_fields_info["input_variable"] = [
    1 if row["variable_field"] in matt.all_variable_fields_input else 0 
    for i, row in df_peru_fields_info.iterrows()
]


df_peru_fields_info.to_csv(
    "20250807_variable_information_table.csv",
    index = None,
    encoding = "UTF-8",
)


In [3]:
path_data_built = pathlib.Path(
    os.path.abspath("../generic_regions/data/output/sisepuede_raw_inputs_latest_PER.csv")
)

In [4]:
### SETUP DIRECTORIES

path_cur = pathlib.Path(os.getcwd())
path_data = path_cur.joinpath("data")
path_transformations = path_cur.joinpath("transformations_peru")


##  SETUP FILES

path_data_calib = path_data.joinpath("input_all_sectors_peru.csv")
path_data_base = path_data.joinpath("real_data_TZA_PER_LKA_20250210.csv")
#path_data_built = path_data.joinpath("sisepuede_raw_global_inputs_peru.csv")
path_data_built = pathlib.Path(
    os.path.abspath("../generic_regions/data/output/sisepuede_raw_inputs_latest_PER.csv")
)

##  SETUP SOME PIPELINE STUFF


file_struct = sfs.SISEPUEDEFileStructure()

# assign some shortcuts
matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)


##  SOME GLOBALS

_REGION_NAME = "peru"
_REGION_ISO = regions.return_region_or_iso(_REGION_NAME, return_type = "iso", )


In [26]:


df_peru = pd.read_csv(path_data_built)
df_peru[regions.key] = _REGION_NAME
df_peru = (
    df_peru[
        df_peru[time_periods.field_time_period].isin(
            time_periods.all_time_periods
        )
    ]
    .drop(
        columns = [time_periods.field_year, regions.field_iso], 
    )
    .reset_index(drop = True, )
)





True

In [ ]:



examples = sxl.SISEPUEDEExamples()
df_examples = examples("input_data_frame").copy()

fields_missing = [x for x in df_examples.columns if x not in df_peru.columns]

df_peru = pd.merge(
    df_peru,
    df_examples[[time_periods.field_time_period] + fields_missing],
    how = "left",
)


In [ ]:
df_peru_new = (
    pd.read_excel(
        "/Users/usuario/Downloads/source_data_frame_2025-08-26.xlsx", 
        skiprows = 7,
    )
)
df_peru_columns = (
    pd.read_excel(
        "/Users/usuario/Downloads/source_data_frame_2025-08-26.xlsx", 
        nrows = 0,
    )
    
)

df_peru_new.columns = df_peru_columns.columns
df_peru_new = df_peru_new.drop(columns = ["Fuente"])

fields_missing = [x for x in df_examples.columns if x not in df_peru_new.columns]
df_peru_new = pd.merge(
    df_peru_new,
    df_examples[[time_periods.field_time_period] + fields_missing],
    how = "left",
)

In [ ]:
modvar_pop_init = matt.get_variable(model_afolu.modvar_lvst_pop_init)
modvar_pop_init.get_from_dataframe(df_peru_new).head()

In [ ]:
modvar_pop_init = matt.get_variable(model_afolu.modvar_lvst_pop_init)
modvar_pop_init.get_from_dataframe(df_examples).head()



In [ ]:
fields = [x for x in df_peru.columns if "lvst" in x and "entferm" in x]
df_peru[fields].head().astype(int)

modvar_pop = matt.get_variable(model_afolu.modvar_lvst_pop)
modvar_pop.get_from_dataframe(df_tmpd)



In [ ]:
df_out_orig[["emission_co2e_subsector_total_lvst"]]

In [ ]:
import sisepuede.models.afolu as mafl
model_afolu = mafl.AFOLU(matt, )
df_out_orig = model_afolu(df_peru, )

In [ ]:
df_out_orig[[x for x in df_out_orig.columns if "emission_co2e" in x and "lvst" in x]]
modvar_pop = matt.get_variable(model_afolu.modvar_lvst_pop)
modvar_pop.get_from_dataframe(df_out_orig)

In [ ]:
import sisepuede.models.afolu as mafl
model_afolu = mafl.AFOLU(matt, )
df_new = model_afolu(df_peru_new, )

In [ ]:
df_new[[x for x in df_new.columns if "emission_co2e" in x and "lvst" in x]]


In [ ]:
#df_peru.to_csv(path_data_built, encoding = "UTF-8", index = None, )




In [242]:
?tmts.trs.tbg.transformation_general

Signature:
tmts.trs.tbg.transformation_general(
    df_input: pandas.core.frame.DataFrame,
    model_attributes: sisepuede.core.model_attributes.ModelAttributes,
    dict_modvar_specs: Dict[str, Dict[str, str]],
    field_region: str = 'nation',
    regions_apply: Optional[List[str]] = None,
    strategy_id: Optional[int] = None,
) -> pandas.core.frame.DataFrame
Docstring:
Generalized function to implement some common transformations. Many other
    transformation functions are wrappers for this function.

Function Arguments
------------------
- df_input: input data frame containing baseline trajectories
- dict_modvar_specs: dictionary mapping model variable to some
    characteristics:

    REQUIRED KEYS
    -------------
    * "magnitude": magnitude of change to apply by final
    * "magnitude_type": type of magnitude to use. Valid types include
        * "baseline_additive": add the magnitude to the baseline
        * "baseline_scalar": multiply baseline value by magnitude
        *

In [244]:


df_tr = tmts.trs.tbg.transformation_general(
    df_peru,
    matt,
    {
        ssp.models.model_enercons.modvar_enfu_exports_fuel: {
            "bounds": (-1, np.inf),
            "categories": ["fuel_natural_gas"],
            "magnitude": 1.8,
            "magnitude_type": "baseline_scalar",
            "vec_ramp": transformers.vec_implementation_ramp
        },
    },
    field_region = "region",
    strategy_id = 18,
)
df_tr["exports_enfu_pj_fuel_natural_gas"]


0     200.951000
1     241.012000
2     228.888000
3     241.012000
4     241.012000
5     241.012000
6     247.438987
7     253.865973
8     260.292960
9     266.719947
10    273.146933
11    279.573920
12    286.000907
13    292.427893
14    298.854880
15    305.281867
16    311.708853
17    318.135840
18    324.562827
19    330.989813
20    337.416800
21    343.843787
22    350.270773
23    356.697760
24    363.124747
25    369.551733
26    375.978720
27    382.405707
28    388.832693
29    395.259680
30    401.686667
31    408.113653
32    414.540640
33    420.967627
34    427.394613
35    433.821600
Name: exports_enfu_pj_fuel_natural_gas, dtype: float64

In [6]:
import sisepuede.transformers as trf
import sisepuede.transformers.transformations as tmts
importlib.reload(tmts.trs)

"""
transformers = trf.Transformers(
    {},
    df_input = df_peru,
)
"""
# initialize a transformations set
if not path_transformations.is_dir():
    trf.instantiate_default_strategy_directory(
        transformers,
        path_transformations,
    )

transformations = tmts.Transformations(
    path_transformations,
    df_input = df_peru,
)

transformers = transformations.transformers

strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)




"""attribute_strategy_new = sf._concat_df(
    [
        strategies.attribute_table.table,
        strategies.build_whirlpool_strategies(6002, )
    ]
)

attribute_strategy_new.to_csv(
    strategies.path_strategy_definition,
    encoding = "UTF-8",
    index = None
)""";



##  WRITE SOME OBJECTS FOR USE WITH CLOUD


sf._write_csv(
    matt.get_dimensional_attribute_table(matt.dim_time_period).table,
    path_transformations.joinpath("attribute_dim_time_period.csv")
)

sf._write_csv(
    df_peru,
    path_transformations.joinpath("input_data_peru_base.csv")
)

In [35]:
strategies.get_strategy(6002).code

'PFLO:ALL'

In [47]:
nm = "lng"
path = f"/Users/usuario/git/julia_districting/julia/comms_{nm}_5.csv"
df_tmp = pd.read_csv(path)
df_tmp = df_tmp[[x for x in df_tmp.columns if len(df_tmp[x].unique()) != 1]]
sf._write_csv(df_tmp, path)

True

In [8]:
tab = strategies.attribute_table.table

# demonstrate A6
strats_run = ["BASE", "PFLO:ALL", "PFLO:NDC_2020"]
strats_run += [x for x in list(tab[strategies.field_strategy_code]) if x.startswith("AGRC:DEC_CH4_RICE")]
strats_run += [x for x in tab["strategy_code"].to_numpy() if x.startswith("PFLO:A6")]
strat_ids_a6 = sorted([strategies.get_strategy(x).id_num for x in strats_run])




In [ ]:
##  Phase 1 for transformations: run transformations one by one using defaults
- start by building all singletons to the templates to run

In [ ]:
df_in[[x for x in df_in.columns if "tag" in x]]
df_in[[x for x in df_in.columns if "min_share" in x]].head(8)


In [30]:

##  GET IN
strat_names = [
    x for x in strategies.attribute_table.table[strategies.field_strategy_name].to_numpy() 
    if x.startswith("Singleton")
]

# init
all_strategies_singleton = [strategies.baseline_id]

# add to list
for strat in strategies.all_strategies:
    nm = strategies.get_strategy(strat).name
    if nm in strat_names:
        all_strategies_singleton.append(strat)




# build templates--trajectory groups are not important right now
examples = sxl.SISEPUEDEExamples()
df_vargroups = examples("variable_trajectory_group_specification")
strategies.build_strategies_to_templates(
    #df_trajgroup = df_vargroups, 
    include_simplex_group_as_trajgroup = False,
    strategies = strat_ids_a6, # all_strategies_singleton,
)


0

In [ ]:
transformers.all_transformers

In [ ]:
##  20251020 - Build transformations for A6

In [ ]:
#### transformations.all_transformation_codes

"""Rice management
Renewable energy
Industrial EE
Electrify LDV
Mode shift freight
Stop deforestation
Increase reforestation
Reduce enteric fermentation
Improve fertilizer application
Reduce HFCs"""

transformations_build = [
    "TX:AGRC:DEC_CH4_RICE",
    "TX:ENTC:TARGET_RENEWABLE_ELEC",
    "TX:INEN:INC_EFFICIENCY_ENERGY",
    "TX:TRNS:SHIFT_FUEL_LIGHT_DUTY",
    "TX:TRNS:SHIFT_MODE_FREIGHT",
    "TX:LNDU:DEC_DEFORESTATION",
    "TX:LNDU:INC_REFORESTATION",
    "TX:LVST:DEC_ENTERIC_FERMENTATION",
    "TX:SOIL:DEC_N_APPLIED",
    "TX:IPPU:DEC_HFCS"
]

##  INITIALIZE KEY ELEMENTS

# some basic vars
delim = "|"
codes = []
descriptions = []
ids = []
names = []
specs = []

#
id_num = max(strategies.attribute_table.key_values) + 1


# reference strategy and associated specifications
dict_strat_builds = {
    "PFLO:ALL": (1, 0), 
    "PFLO:NDC_2020": (1, 1),
}
dict_strat_pack = {}


#
strats_to_build = ["PFLO:ALL", "PFLO:NDC_2020"]
for code in strats_to_build:
    # get transformaion information
    strat_ref = strategies.get_strategy(code, )
    all_transformations_ref = strat_ref.get_transformation_list(
        strat_ref.transformation_specification,
        transformations,
    )
    all_transformation_codes_ref = [x.code for x in all_transformations_ref]

    # add to dict
    tup = (strat_ref, all_transformations_ref, all_transformation_codes_ref, )
    dict_strat_pack.update({code: tup, })



# map transformer codes to all transformations
dict_transformer_code_to_transformations = transformations.get_transformation_codes_by_transformer_code()




##  ITERATE OVER TRANSFORMATIONS THAT NEED TO BE ADDED/REMOVED FROM BASE REF

for strat_build, strat_pack in dict_strat_pack.items():

    builds = dict_strat_builds.get(strat_build)
    (strat_ref, all_transformations_ref, all_transformation_codes_ref, ) = strat_pack

    
    for transformation_code in transformations_build:
    
        # check for transformations in NDC
        transformation = transformations.get_transformation(transformation_code)
        if transformation is None:
            raise RuntimeError(f"Transformation code {transformation_code} not found.")
    
    
        # get transformer and check for existing transformation in reference strategy with that code
        all_transformations_with_code = dict_transformer_code_to_transformations.get(
            transformation.transformer_code, 
        )
    
        tfmns_with_trfr_in_strat_ref = set(all_transformations_with_code) & set(all_transformation_codes_ref)
        if len(tfmns_with_trfr_in_strat_ref) == 0: continue
    
        # transformation to replace with current one (for 100%) and remove (for 0%)
        transformation_remove = list(tfmns_with_trfr_in_strat_ref)[0]
    
    
        ## build transformation WITH 100% implementation
        transformation_specification_0 = sorted([x for x in all_transformation_codes_ref if x != transformation_remove])
        transformation_specification_100 = sorted(transformation_specification_0 + [transformation_code])
        
        transformation_specification_0_str = delim.join(transformation_specification_0)
        transformation_specification_100_str = delim.join(transformation_specification_100)

        # build the 0?
        if builds[0] == 1:
            code = transformation_remove.replace(":", "")
            code = f"{strat_build}_NO_{code}"
            code = code.replace("PFLO:", "PFLO:A6_")
            name = f"A6 {strat_build} without {transformation_remove}"

            # append
            codes.append(code)
            descriptions.append(f"{name} for A6")
            ids.append(id_num)
            names.append(name)
            specs.append(transformation_specification_0_str)

            id_num += 1


        # build the 100?
        if builds[1] == 1:
            code = transformation_code.replace(":", "")
            code = f"{strat_build}_WITH_{code}"
            code = code.replace("PFLO:", "PFLO:A6_")
            name = f"A6 {strat_build} with {transformation_code}"

            # append
            codes.append(code)
            descriptions.append(f"{name} replacing {transformation_remove} for A6")
            ids.append(id_num)
            names.append(name)
            specs.append(transformation_specification_100_str)

            id_num += 1




df_additional_strategies = pd.DataFrame(
    {
        strategies.attribute_table.key: ids,
        strategies.field_baseline_strategy: np.zeros(len(specs)).astype(int),
        strategies.field_description: descriptions,
        strategies.field_strategy_code: codes,
        strategies.field_strategy_name: names,
        strategies.field_transformation_specification: specs,
    }
)




In [ ]:
attr_strat_out = strategies.attribute_table.table
attr_strat_out = sf._concat_df(
    [
        attr_strat_out,
        df_additional_strategies[attr_strat_out.columns]
    ]
)

In [ ]:
"""sf._write_csv(
    attr_strat_out,
    strategies.path_strategy_definition,
)"""

In [ ]:
# Next, build a SISEPUEDE object and load strategies

In [7]:
import sisepuede as si

ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    #id_str = "sisepuede_run_2025-10-23T16:31:38.101072",
    initialize_as_dummy = False,
    regions = [_REGION_NAME],
    strategies = strategies,
    #try_exogenous_xl_types_in_variable_specification = True,
)





2025-11-13 02:58:47,885 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-11-13 02:58:47,886 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-11-13 02:58:47,887 - INFO - 	Setting export engine to 'csv'.
2025-11-13 02:58:47,887 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2025-11-13 02:58:47,887 - INFO - Successfully instantiated table ANALYSIS_METADATA
2025-11-13 02:58:47,888 - WARNING - No index fields found in ATTRIBUTE_DESIGN. Initializing index fields.
2025-11-13 02:58:47,888 - INFO - Successfully instantiated table ATTRIBUTE_DESIGN
2025-11-13 02:58:47,888 - WARNING - No index fields found in ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES. Initializing index fields.
2025-11-13 02:58:47,889 - INFO - Successfully instantiated table ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES
2025-11-13 02:58:47,889 - WARNING - No index fi

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   2032.8 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
2025-11-13 02:59:52,534 - INFO - Successfully initialized JuMP optimizer from solver module HiGHS.
2025-11-13 02:59:52,549 - INFO - Successfully initialized SISEPUEDEModels.
2025-11-13 02:59:52,555 - INFO - Table ANALYSIS_METADATA successfully written to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_ru

In [ ]:
tx_all = "TX:AGRC:DEC_CH4_RICE_NDC2020_EST|TX:AGRC:DEC_EXPORTS_NDC2020_EST|TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN_NDC2020_EST|TX:AGRC:INC_CONSERVATION_AGRICULTURE_NDC2020_EST|TX:AGRC:INC_PRODUCTIVITY_NDC2020_EST|TX:CCSQ:INC_CAPTURE_NDC2020_EST|TX:ENTC:DEC_LOSSES_NDC2020_EST|TX:ENTC:TARGET_CLEAN_HYDROGEN_NDC2020_EST|TX:ENTC:TARGET_RENEWABLE_ELEC_NDC2020_EST|TX:FGTV:DEC_LEAKS_NDC2020_EST|TX:FGTV:INC_FLARE_NDC2020_EST|TX:INEN:INC_EFFICIENCY_ENERGY_NDC2020_EST|TX:INEN:INC_EFFICIENCY_PRODUCTION_NDC2020_EST|TX:INEN:SHIFT_FUEL_HEAT_NDC2020_EST|TX:IPPU:DEC_CLINKER_NDC2020_EST|TX:IPPU:DEC_DEMAND_NDC2020_EST|TX:IPPU:DEC_HFCS_NDC2020_EST|TX:IPPU:DEC_N2O_NDC2020_EST|TX:IPPU:DEC_OTHER_FCS_NDC2020_EST|TX:IPPU:DEC_PFCS_NDC2020_EST|TX:LNDU:DEC_DEFORESTATION_NDC2020_EST|TX:LNDU:DEC_SOC_LOSS_PASTURES_NDC2020_EST|TX:LNDU:INC_REFORESTATION_NDC2020_EST|TX:LNDU:INC_SILVOPASTURE_NDC2020_EST|TX:LNDU:PLUR_NDC2020_EST|TX:LSMM:INC_CAPTURE_BIOGAS_NDC2020_EST|TX:LVST:DEC_ENTERIC_FERMENTATION_NDC2020_EST|TX:LVST:DEC_EXPORTS_NDC2020_EST|TX:LVST:INC_PRODUCTIVITY_NDC2020_EST|TX:SCOE:DEC_DEMAND_HEAT_NDC2020_EST|TX:SCOE:INC_EFFICIENCY_APPLIANCE_NDC2020_EST|TX:SCOE:SHIFT_FUEL_HEAT_NDC2020_EST|TX:SOIL:DEC_LIME_APPLIED_NDC2020_EST|TX:SOIL:DEC_N_APPLIED_NDC2020_EST|TX:TRDE:DEC_DEMAND_NDC2020_EST|TX:TRNS:INC_EFFICIENCY_ELECTRIC_NDC2020_EST|TX:TRNS:INC_EFFICIENCY_NON_ELECTRIC_NDC2020_EST|TX:TRNS:INC_OCCUPANCY_LIGHT_DUTY_NDC2020_EST|TX:TRNS:SHIFT_FUEL_LIGHT_DUTY_NDC2020_EST|TX:TRNS:SHIFT_FUEL_MARITIME_NDC2020_EST|TX:TRNS:SHIFT_FUEL_MEDIUM_DUTY_NDC2020_EST|TX:TRNS:SHIFT_FUEL_RAIL_NDC2020_EST|TX:TRNS:SHIFT_MODE_FREIGHT_NDC2020_EST|TX:TRNS:SHIFT_MODE_PASSENGER_NDC2020_EST|TX:TRWW:INC_CAPTURE_BIOGAS_NDC2020_EST|TX:TRWW:INC_COMPLIANCE_SEPTIC_NDC2020_EST|TX:WALI:INC_TREATMENT_RURAL_NDC2020_EST|TX:WALI:INC_TREATMENT_URBAN_NDC2020_EST|TX:WASO:DEC_CONSUMER_FOOD_WASTE_NDC2020_EST|TX:WASO:INC_ANAEROBIC_AND_COMPOST_NDC2020_EST|TX:WASO:INC_CAPTURE_BIOGAS_NDC2020_EST|TX:WASO:INC_ENERGY_FROM_BIOGAS_NDC2020_EST|TX:WASO:INC_ENERGY_FROM_INCINERATION_NDC2020_EST|TX:WASO:INC_LANDFILLING_NDC2020_EST|TX:WASO:INC_RECYCLING_NDC2020_EST"
tx_a6 = "TX:AGRC:DEC_CH4_RICE_P10|TX:AGRC:DEC_EXPORTS_NDC2020_EST|TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN_NDC2020_EST|TX:AGRC:INC_CONSERVATION_AGRICULTURE_NDC2020_EST|TX:AGRC:INC_PRODUCTIVITY_NDC2020_EST|TX:CCSQ:INC_CAPTURE_NDC2020_EST|TX:ENTC:DEC_LOSSES_NDC2020_EST|TX:ENTC:TARGET_CLEAN_HYDROGEN_NDC2020_EST|TX:ENTC:TARGET_RENEWABLE_ELEC_NDC2020_EST|TX:FGTV:DEC_LEAKS_NDC2020_EST|TX:FGTV:INC_FLARE_NDC2020_EST|TX:INEN:INC_EFFICIENCY_ENERGY_NDC2020_EST|TX:INEN:INC_EFFICIENCY_PRODUCTION_NDC2020_EST|TX:INEN:SHIFT_FUEL_HEAT_NDC2020_EST|TX:IPPU:DEC_CLINKER_NDC2020_EST|TX:IPPU:DEC_DEMAND_NDC2020_EST|TX:IPPU:DEC_HFCS_NDC2020_EST|TX:IPPU:DEC_N2O_NDC2020_EST|TX:IPPU:DEC_OTHER_FCS_NDC2020_EST|TX:IPPU:DEC_PFCS_NDC2020_EST|TX:LNDU:DEC_DEFORESTATION_NDC2020_EST|TX:LNDU:DEC_SOC_LOSS_PASTURES_NDC2020_EST|TX:LNDU:INC_REFORESTATION_NDC2020_EST|TX:LNDU:INC_SILVOPASTURE_NDC2020_EST|TX:LNDU:PLUR_NDC2020_EST|TX:LSMM:INC_CAPTURE_BIOGAS_NDC2020_EST|TX:LVST:DEC_ENTERIC_FERMENTATION_NDC2020_EST|TX:LVST:DEC_EXPORTS_NDC2020_EST|TX:LVST:INC_PRODUCTIVITY_NDC2020_EST|TX:SCOE:DEC_DEMAND_HEAT_NDC2020_EST|TX:SCOE:INC_EFFICIENCY_APPLIANCE_NDC2020_EST|TX:SCOE:SHIFT_FUEL_HEAT_NDC2020_EST|TX:SOIL:DEC_LIME_APPLIED_NDC2020_EST|TX:SOIL:DEC_N_APPLIED_NDC2020_EST|TX:TRDE:DEC_DEMAND_NDC2020_EST|TX:TRNS:INC_EFFICIENCY_ELECTRIC_NDC2020_EST|TX:TRNS:INC_EFFICIENCY_NON_ELECTRIC_NDC2020_EST|TX:TRNS:INC_OCCUPANCY_LIGHT_DUTY_NDC2020_EST|TX:TRNS:SHIFT_FUEL_LIGHT_DUTY_NDC2020_EST|TX:TRNS:SHIFT_FUEL_MARITIME_NDC2020_EST|TX:TRNS:SHIFT_FUEL_MEDIUM_DUTY_NDC2020_EST|TX:TRNS:SHIFT_FUEL_RAIL_NDC2020_EST|TX:TRNS:SHIFT_MODE_FREIGHT_NDC2020_EST|TX:TRNS:SHIFT_MODE_PASSENGER_NDC2020_EST|TX:TRWW:INC_CAPTURE_BIOGAS_NDC2020_EST|TX:TRWW:INC_COMPLIANCE_SEPTIC_NDC2020_EST|TX:WALI:INC_TREATMENT_RURAL_NDC2020_EST|TX:WALI:INC_TREATMENT_URBAN_NDC2020_EST|TX:WASO:DEC_CONSUMER_FOOD_WASTE_NDC2020_EST|TX:WASO:INC_ANAEROBIC_AND_COMPOST_NDC2020_EST|TX:WASO:INC_CAPTURE_BIOGAS_NDC2020_EST|TX:WASO:INC_ENERGY_FROM_BIOGAS_NDC2020_EST|TX:WASO:INC_ENERGY_FROM_INCINERATION_NDC2020_EST|TX:WASO:INC_LANDFILLING_NDC2020_EST|TX:WASO:INC_RECYCLING_NDC2020_EST"

l_all = tx_all.split("|")
l_a6 = tx_a6.split("|")

set(l_all) - set(l_a6)


In [ ]:
dict_strat_to_prim = sf.build_dict(
    ssp.odpt_primary.get_indexing_dataframe_from_primary_key(
        df_out[ssp.key_primary].unique()
    )
    .get(
        [
            ssp.key_strategy,
            ssp.key_primary,
        ]
    )
)

flds_all = [x for x in df_out.columns if "emission_co2e_subsector_total_" in x]
for i in range(1, 10):
    s0 = "PFLO:ALL"#"PFLO:NDC_2020"
    s1 = f"PFLO:A6_ALL_WITH_RICE_CH4_P{i}0"
    
    
    prim0 = dict_strat_to_prim.get(strategies.get_strategy(s0).id_num)
    prim1 = dict_strat_to_prim.get(strategies.get_strategy(s1).id_num)
    
    # base cases
    dfi0 = df_in[
       df_in[ssp.key_primary].isin([prim0]) 
    ]
    df0 = df_out[
       df_out[ssp.key_primary].isin([prim0]) 
    ]
    vec0 = df0[flds_all].iloc[-1].to_numpy()
    val0 = vec0.sum()

    # a6 cases
    dfi1 = df_in[
       df_in[ssp.key_primary].isin([prim1]) 
    ]
    df1 = df_out[
       df_out[ssp.key_primary].isin([prim1]) 
    ]
    vec1 = df1[flds_all].iloc[-1].to_numpy()
    val1 = vec1.sum()

    for i, val in enumerate(vec0):
        print(f"checking {i}...")
        val_compare = vec1[i]
        if val != val_compare:
            fld = flds_all[i]
            print(f"diff found in field {fld} at {i}")

    print(f"val NDC:\t\t{val0}\nval at percentile {i}:\t{val1}\n")




In [ ]:
flds_entc = [x for x in df0.columns if "emission_co2e" in x and "entc" in x]
flds_waso = [x for x in df0.columns if "emission_co2e" in x and "waso" in x]

"""
for f in df0.columns:
    if float(df0[f].iloc[-1]) != float(df1[f].iloc[-1]):
        print(f)
"""


for f in dfi0.columns:
    if f in ["region"]: continue
    if float(dfi0[f].iloc[-1]) != float(dfi1[f].iloc[-1]):
        print(f)

In [ ]:
df_

In [ ]:
strat_base = strategies.get_strategy(s0)
strat_a6 = strategies.get_strategy(s1)

set_base = strat_base.transformation_specification.split("|")
set_a6 = strat_a6.transformation_specification.split("|")

set(set_a6) - set(set_base)

In [ ]:
df_tmp0 = ssp.models(strat_base(), include_electricity_in_energy = True, )
df_tmp1 = ssp.models(strat_a6(), include_electricity_in_energy = True, )



In [ ]:
for f in df_tmp0.columns:
    if f in ["region"]: continue
    if float(df_tmp0[f].iloc[-1]) != float(df_tmp1[f].iloc[-1]):
        print(f)

In [ ]:
df_tmp0[flds_all].sum(axis = 1).iloc[-1]

In [ ]:
df_out[
   df_out[ssp.key_primary].isin([prim1]) 
][[x for x in df_out.columns if "emission_co2e_subsector_total" in x]].sum(axis = 1).tail()

In [ ]:
fields = [x for x in df_in.columns if ("ch4" in x) and ("agrc" in x) and ("rice" in x)]
df_in[
   df_in[ssp.key_primary].isin([prim1]) 
][fields]




In [ ]:
tab = ssp.database.db.dict_iterative_database_tables.get("MODEL_OUTPUT")
pathlib.Path(tab.fp_table).is_file()

In [ ]:
primary_keys = dict_run
check_results = True
chunk_size = 10
force_overwrite_existing_primary_keys = False
max_attempts = 2
regions = None
reinitialize_output_table_on_verification_failure = False
save_inputs = True
skip_nas_in_input = False
max_attempts = (
    int(min(max(max_attempts, 1), 5))
    if sf.isnumber(max_attempts)
    else 2
)
save_inputs = (
    ssp.get_config_parameter("save_inputs")
    if save_inputs is None
    else save_inputs
)

# get all scenarios and designs associated with them
primary_keys = ssp.get_primary_keys(primary_keys)
df_primary_keys = ssp.odpt_primary.get_indexing_dataframe(
    key_values = primary_keys
)
all_designs = sorted(list(set(df_primary_keys[ssp.key_design])))

# initializations
df_out = []
df_out_inputs = []
df_out_primary = []
dict_primary_keys_run = dict((x, [None for x in primary_keys]) for x in ssp.regions)
iterate_outer = 0

# available indices and resolution
idt = ssp.database.db.dict_iterative_database_tables.get(
    ssp.database.table_name_output
)
index_conflict_resolution = None
index_conflict_resolution = (
    "write_replace" 
    if (force_overwrite_existing_primary_keys or (idt.index_conflict_resolution == "write_replace")) 
    else None
)
set_available_ids = idt.available_indices

# check regions specification
regions = (
    [x for x in ssp.regions if x in regions]
    if sf.islistlike(regions)
    else None
)
regions = ssp.regions if (regions is None) else regions



region = regions[0]

iterate_inner = 0

# retrieve region specific future trajectories and lhs design
future_trajectories_cur = ssp.experimental_manager.dict_future_trajectories.get(region)
lhs_design_cur = ssp.experimental_manager.dict_lhs_design.get(region)
region_out = ssp.get_output_region(region)

#self._log(f"\n***\tSTARTING REGION {region}\t***\n", type_log = "info")



design = all_designs[0]

df_lhs_l, df_lhs_x = lhs_design_cur.retrieve_lhs_tables_by_design(
    design,
    return_type = pd.DataFrame
)

# get reduced set of primary keys
df_primary_keys_cur_design = df_primary_keys[
    df_primary_keys[ssp.key_design] == design
]
keep_futures = sorted(list(set(df_primary_keys_cur_design[ssp.key_future])))

# reduce lhs tables - LEs
df_lhs_l = (
    df_lhs_l[
        df_lhs_l[ssp.key_future].isin(keep_futures)
    ] 
    if (df_lhs_l is not None) 
    else df_lhs_l
)
# Xs
df_lhs_x = (
    df_lhs_x[
        df_lhs_x[ssp.key_future].isin(keep_futures)
    ] 
    if (df_lhs_x is not None) 
    else df_lhs_x
)



future = keep_futures[0]


# determine if baseline future and fetch lhs rows
base_future_q = (future == ssp.baseline_future)
lhs_l = (
    df_lhs_l[df_lhs_l[ssp.key_future] == future].iloc[0] 
    if ((df_lhs_l is not None) and not base_future_q) 
    else None
)
lhs_x = (
    df_lhs_x[df_lhs_x[ssp.key_future] == future].iloc[0] 
    if ((df_lhs_x is not None) and not base_future_q) 
    else None
)

# generate the futures and get available strategies
df_input = future_trajectories_cur.generate_future_from_lhs_vector(
    lhs_x,
    df_row_lhc_sample_l = lhs_l,
    future_id = future,
    baseline_future_q = base_future_q
)
all_strategies = sorted(list(
    set(df_input[ssp.key_strategy])
))




strategy = all_strategies[0]
# get primary id info
df_primary_keys_cur_design_fs = (
    df_primary_keys_cur_design[
        df_primary_keys_cur_design[ssp.key_future].isin([future]) &
        df_primary_keys_cur_design[ssp.key_strategy].isin([strategy])
    ]
    .reset_index(drop = True)
)

id_primary = df_primary_keys_cur_design_fs[ssp.key_primary]
id_primary = int(id_primary.iloc[0]) if (len(id_primary) > 0) else None
write_q = ((region_out, id_primary) not in set_available_ids) 
write_q |= (index_conflict_resolution == "write_replace")






In [ ]:
tab = ssp.database.db.dict_iterative_database_tables.get("MODEL_INPUT")

In [ ]:
tab = ssp.database.db.dict_iterative_database_tables.get("MODEL_OUTPUT")


In [ ]:
## then, run the strategies one by one

In [9]:



dict_run = {
    ssp.key_future: [0],
    ssp.key_design: [0],
    ssp.key_strategy: strat_ids_a6,#[0:1],
}

#primaries = [186186, 187187, 188188, 189189]
# we'll save inputs since we're doing a small set of runs 
dict_successful = ssp(
    dict_run, 
    force_overwrite_existing_primary_keys = True,
    save_inputs = True,
)

2025-11-13 02:59:59,492 - INFO - 
***	STARTING REGION peru	***

2025-11-13 03:00:01,164 - INFO - Trying run primary_id = 0 in region peru
2025-11-13 03:00:01,165 - INFO - Running AFOLU model
2025-11-13 03:00:01,406 - INFO - AFOLU model run successfully completed
2025-11-13 03:00:01,406 - INFO - Running CircularEconomy model
2025-11-13 03:00:01,432 - INFO - CircularEconomy model run successfully completed
2025-11-13 03:00:01,433 - INFO - Running IPPU model
2025-11-13 03:00:01,476 - INFO - IPPU model run successfully completed
2025-11-13 03:00:01,477 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-11-13 03:00:01,487 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2025-11-13 03:00:01,539 - INFO - EnergyConsumption without Fugitive Emissions model run successfully completed
2025-11-13 03:00:01,539 - INFO - Running Energy model (Electricity and Fuel Production: trying to ca

2025-13-Nov 03:00:01.830 Opened SQLite database at /Users/usuario/git/sisepuede/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2025-13-Nov 03:00:01.975 Added NEMO structure to SQLite database at /Users/usuario/git/sisepuede/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2025-13-Nov 03:00:17.358 Started modeling scenario. NEMO version = 2.2.0, solver = HiGHS.


2025-11-13 03:02:05,629 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2025-11-13 03:02:05,636 - INFO - EnergyProduction model run successfully completed
2025-11-13 03:02:05,637 - INFO - Running Energy (Fugitive Emissions)
2025-11-13 03:02:05,657 - INFO - Fugitive Emissions from Energy model run successfully completed
2025-11-13 03:02:05,658 - INFO - Appending Socioeconomic outputs
2025-11-13 03:02:05,662 - INFO - Socioeconomic outputs successfully appended.
2025-11-13 03:02:05,665 - INFO - Model run for primary_id = 0 successfully completed in 124.5 seconds (n_tries = 1).
2025-11-13 03:02:05,668 - INFO - Trying run primary_id = 1001 in region peru
2025-11-13 03:02:05,669 - INFO - Running AFOLU model


2025-13-Nov 03:00:17.831 Started optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-13-Nov 03:00:40.339 Finished optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-13-Nov 03:00:40.386 Started optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
2025-13-Nov 03:02:05.511 Finished optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
set ['emission_co2e_co2_entc_bmass_processing_and_refinement_fp_hydrogen_gasification'] = 0 in energy production. FIX WITH NEW fp_hydrogen_gasification_biomass TECH.


2025-11-13 03:02:05,902 - INFO - AFOLU model run successfully completed
2025-11-13 03:02:05,902 - INFO - Running CircularEconomy model
2025-11-13 03:02:05,926 - INFO - CircularEconomy model run successfully completed
2025-11-13 03:02:05,926 - INFO - Running IPPU model
2025-11-13 03:02:05,966 - INFO - IPPU model run successfully completed
2025-11-13 03:02:05,966 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2025-11-13 03:02:05,977 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2025-11-13 03:02:06,024 - INFO - EnergyConsumption without Fugitive Emissions model run successfully completed
2025-11-13 03:02:06,025 - INFO - Running Energy model (Electricity and Fuel Production: trying to call Julia)


...

2025-11-13 07:27:28,247 - INFO - Table MODEL_OUTPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_OUTPUT.csv.
2025-11-13 07:27:28,642 - INFO - Table MODEL_INPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_INPUT.csv.
2025-11-13 07:27:28,643 - INFO - Trying run primary_id = 200200 in region peru
2025-11-13 07:27:28,644 - INFO - Running AFOLU model
2025-11-13 07:27:28,866 - INFO - AFOLU model run successfully completed
2025-11-13 07:27:28,867 - INFO - Running CircularEconomy model
2025-11-13 07:27:28,889 - INFO - CircularEconomy model run successfully completed
2025-11-13 07:27:28,890 - INFO - Running IPPU model
2025-11-13 07:27:28,928 - INFO - IPPU model run successfully completed
2025-11-13 07:27:28,928 - INFO - Running Energy model (Ene

2025-13-Nov 07:27:29.601 Started modeling scenario. NEMO version = 2.2.0, solver = HiGHS.
2025-13-Nov 07:27:29.647 Started optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].


2025-11-13 07:28:53,903 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2025-11-13 07:28:53,909 - INFO - EnergyProduction model run successfully completed
2025-11-13 07:28:53,909 - INFO - Running Energy (Fugitive Emissions)
2025-11-13 07:28:53,931 - INFO - Fugitive Emissions from Energy model run successfully completed
2025-11-13 07:28:53,931 - INFO - Appending Socioeconomic outputs
2025-11-13 07:28:53,936 - INFO - Socioeconomic outputs successfully appended.
2025-11-13 07:28:53,938 - INFO - Model run for primary_id = 200200 successfully completed in 85.29 seconds (n_tries = 1).
2025-11-13 07:28:53,939 - INFO - 
***	 REGION peru COMPLETE	***

2025-11-13 07:28:54,011 - INFO - Table MODEL_OUTPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_OUTPUT.csv.
2025-11-13 07:28:54,013 - INFO - Table ATTRIBUTE_PRIMARY successfully written to /Users/usua

2025-13-Nov 07:27:47.173 Finished optimizing following years: [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011].
2025-13-Nov 07:27:47.205 Started optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
2025-13-Nov 07:28:53.813 Finished optimizing following years: [1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035].
set ['emission_co2e_co2_entc_bmass_processing_and_refinement_fp_hydrogen_gasification'] = 0 in energy production. FIX WITH NEW fp_hydrogen_gasification_biomass TECH.


2025-11-13 07:28:54,119 - INFO - Table MODEL_INPUT successfully appended to /Users/usuario/git/sisepuede/sisepuede/out/sisepuede_run_2025-11-13T02;58;47.555474/sisepuede_run_2025-11-13T02;58;47.555474_output_database/MODEL_INPUT.csv.


In [295]:
primaries = dict_successful.get("peru")


In [301]:
##  Read the outputs and inputs back in
attribute_prim = ssp.odpt_primary.get_indexing_dataframe_from_primary_key(primaries)
attribute_prim[attribute_prim["strategy_id"].isin(range(1000, 1010))]

,primary_id,design_id,strategy_id,future_id
1,1001,0,1000,0
2,2002,0,1001,0
3,3003,0,1002,0
4,4004,0,1003,0
5,5005,0,1004,0
6,6006,0,1005,0
7,7007,0,1006,0
8,8008,0,1007,0
9,9009,0,1008,0
10,10010,0,1009,0


In [310]:
flds = ssp.model_attributes.get_all_subsector_emission_total_fields()
df_out_tmp = df_out.copy()
field_total = "net_emission_co2e"
df_out_tmp[field_total] = df_out_tmp[flds].sum(axis = 1)
df_tots = (
    df_out_tmp[
        df_out_tmp[ssp.key_time_period].isin([35])
    ]
    .get([ssp.key_primary, field_total])
    .reset_index(drop = True)
)

df_tots[
    df_tots[ssp.key_primary] <= 10010
]

,primary_id,net_emission_co2e
0,0,365.643334
1,1001,364.904697
2,2002,365.569471
3,3003,365.495607
4,4004,365.421743
5,5005,365.347880
6,6006,365.232980
7,7007,365.200152
8,8008,365.126288
9,9009,365.052425


In [11]:
df_out_tmp = ssp.read_output(None)
df_in_tmp = ssp.read_input(None)

In [16]:
?sf.check_row_sums

Signature:
sf.check_row_sums(
    array: numpy.ndarray,
    axis: int = 1,
    msg_pass: str = '',
    sum_restriction: Union[float, numpy.ndarray] = 1,
    thresh_correction: Optional[float] = 0.001,
) -> numpy.ndarray
Docstring:
Check row sums to ensure they add to sum_restriction

Function Arguments
------------------
array : np.ndarray
    Array to check sums against

Keyword Arguments
-----------------
axis : int
    Axis of array to sum over
msg_pass : str
    Optional message to pass to error
sum_restriction : Union[float, np.ndarray]
    Float or NumPy array with same shape as axis, used to restrict sums
thresh_correction : Union[float, None]
    Threshold for correction. If None, will correct any value
File:      ~/git/sisepuede/sisepuede/utilities/_toolbox.py
Type:      function

In [10]:
ssp.generate_summary_files(None, export = True, );











In [ ]:
df_long = pd.read_csv("/Users/usuario/Downloads/cb_data_long.csv")

In [ ]:
import sisepuede.visualization.plots as svp
svp.plot_emissions_stack(df_out, matt, )

In [ ]:
# df_from_pipeline.columns

In [ ]:
# Bulid Dataset for 202506 model with data we have

- Pull data fromo original repo (df_peru_base)
- Update with latest data (from new repo)
- Fill in values for new vars using `temp_update_fields_from_wv_to_main`
- Check if anything is missing

In [ ]:
importlib.reload(dc)
construct = dc.SISEPUEDEDataConstructs(
    path_output_database = "/Users/usuario/git/sisepuede_data_pipeline/sisepuede_inputs.sqlite",
)

path_repo = pathlib.Path("/Users/usuario/SISEPUEDE_DATA_REPOSITORY")
repo = pu.Repository(
    {
        "local": {
            "path": str(path_repo)
        }
    }
)

In [ ]:
import temp_update_fields_from_wv_to_main as temp 

examples = sxl.SISEPUEDEExamples()
df_example_input = examples("input_data_frame")

# get from original repo
df_peru_base = (
    pd.read_csv(path_data_base)
    .rename(
        columns = {
            "iso_code3": regions.field_iso,
            "Year": time_periods.field_year,
        }
    )
)

# get from pipeline
df_from_pipeline = construct.build_inputs_from_database(
    regions_keep = [_REGION_NAME],
)


# add in pipeline-based data
df_peru = sf.match_df_to_target_df(
    df_peru_base[
        df_peru_base[regions.field_iso].isin([_REGION_ISO])
    ],
    
    df_from_pipeline
    .drop(columns = [regions.field_iso]),
    
    [
        construct.time_periods.field_year,
    ],
    overwrite_only = False,
)

##  PULL FROM EXAMPLE DF

# fields not in peru
fields_missing = [
    x for x in df_example_input.columns 
    if (x not in df_peru.columns) 
    and (x in matt.all_variable_fields_input)
]

# specify fields to pull from the example
fields_from_ex = [
    x for x in fields_missing 
    if not (
        x.startswith("factor_lndu")
        or x.startswith("frac_lndu_")
    )
]

fields_from_ex = [
    x for x in fields_from_ex
    if (x not in df_peru.columns)
]

# merge in from ex
df_peru = pd.merge(
    df_peru,
    df_example_input[fields_from_ex + [time_periods.field_time_period]],
)


##  TEMPORARY SCRIPT FOR MOVING FROM working_version TO latest full version

df_peru = temp.update_fields(
    df_peru,
    matt,
)

# add in key
df_peru[regions.key] = _REGION_NAME


In [ ]:
[x for x in dir(matt) if "get" in x and "emission" in x]

In [ ]:
# Try running to make sure it works and looks reasonable

In [ ]:
models = sm.SISEPUEDEModels(
    matt,
    fp_julia = file_struct.dir_jl,
    fp_nemomod_reference_files = file_struct.dir_ref_nemo,
)
df_run_enerprod = models(df_peru, include_electricity_in_energy = True, )

In [ ]:
df_run = models(df_peru, include_electricity_in_energy = True, )

In [ ]:
import sisepuede.plotting.plots as spp

In [ ]:
modvar = matt.get_variable("Land Use Area", )
fig, ax = plt.subplots(figsize = (12, 8))
df_run_enerprod[modvar.fields].plot.area(cmap = "tab20", ax = ax, )
ax.legend(loc = "upper right")


In [ ]:
vals = matt.get_attribute_table("Liquid Waste").key_values
for k in vals:
    print(k)

In [ ]:
fig, ax = plt.subplots(figsize = (11, 7))
spp.plot_emissions_stack(
    df_run_enerprod,
    matt,
    figtuple = (fig, ax),
)

ax.legend()


In [ ]:
# Skip here to run model

In [ ]:
df_peru.to_csv(
    dir_data.joinpath("sisepuede_raw_global_inputs_peru.csv"),
    encoding = "UTF-8",
    index = None,
)

In [ ]:
skip = 0


In [ ]:
import sisepuede.transformers.transformers as trf2
importlib.reload(trf2)

transformers = trf2.Transformers(
    {},
    df_input = df_peru,
)



In [ ]:
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)

all_primaries = sorted(list(df_out[ssp.key_primary].unique()))

# build if unable to simply read the data frame
if df_in is None:
    df_in = []
     
    for region in ssp.regions:
        for primary in all_primaries: 
            df_in_filt = ssp.generate_scenario_database_from_primary_key(primary)
            df_in.append(df_in_filt.get(region))
    
    df_in = pd.concat(df_in, axis = 0).reset_index(drop = True)


df_export = pd.merge(
    df_out,
    df_in,
    how = "left",
)





# check output directory 
dir_pkg = os.path.join(
    ssp.file_struct.dir_out, 
    f"sisepuede_summary_results_run_{ssp.id_fs_safe}"
)
os.makedirs(dir_pkg) if not os.path.exists(dir_pkg) else None


for tab in ["ATTRIBUTE_STRATEGY"]:
    ssp.database.db.read_table(tab).to_csv(
        os.path.join(dir_pkg, f"{tab}.csv"),
        index = None,
        encoding = "UTF-8"
    )


df_primary = (
    ssp
    .odpt_primary
    .get_indexing_dataframe(
        sorted(list(df_out[ssp.key_primary].unique()))
    )
)
    
df_primary.to_csv(
    os.path.join(dir_pkg, f"ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

(
    ssp.model_attributes.get_dimensional_attribute_table(
        ssp.model_attributes.dim_time_period
    )
    .to_csv(
        os.path.join(dir_pkg, f"ATTRIBUTE_TIME_PERIOD.csv"),
    )
)

df_export.to_csv(
    os.path.join(dir_pkg, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

In [ ]:
import sisepuede.models.afolu as mafl

In [ ]:
dir_pkg

In [ ]:
df_afolu = pd.read_excel("/Users/usuario/Downloads/Var_SSP_AFOLU_2.xlsx")
df_afolu["time_period"] = range(36)

## ADJUST GENERATION FACTORS TO BE UNITLESS RATIOS

# get vars
modvar_gf_nitrogen = matt.get_variable("Daily Nitrogen Generation Factor")
modvar_gf_volsol = matt.get_variable("Daily Volatile Solid Generation Factor")
modvar_animal_weight = matt.get_variable("Animal Weight")

# get fields
cats_lvst = ["cattle_dairy", "cattle_nondairy"]
fields_aw = modvar_animal_weight.build_fields(category_restrictions = cats_lvst)
fields_n = modvar_gf_nitrogen.build_fields(category_restrictions = cats_lvst)
fields_vs = modvar_gf_volsol.build_fields(category_restrictions = cats_lvst)

# normalize by animal weight
arr_mass = df_peru[fields_aw].to_numpy()
arr_n_new = df_afolu[fields_n].to_numpy()/arr_mass
arr_vs_new = df_afolu[fields_vs].to_numpy()/arr_mass

# overwrite
df_afolu[fields_n] = arr_n_new
df_afolu[fields_vs] = arr_vs_new



df_peru_tmp = sf.match_df_to_target_df(
    df_peru,
    df_afolu[["time_period"] + [x for x in df_afolu.columns if ("lvst" in x) or ("lsmm" in x)]],
    ["time_period"],
    overwrite_only = True,
)




# get missing fields from example
examples = sxl.SISEPUEDEExamples()
df_example_input = examples("input_data_frame").copy()

# fields not in peru
fields_missing = [
    x for x in df_example_input.columns 
    if (x not in df_peru_tmp.columns) 
    and (x in matt.all_variable_fields_input)
]


# merge in from ex
df_peru_tmp = pd.merge(
    df_peru_tmp,
    df_example_input[fields_missing + [time_periods.field_time_period]],
)






df_tmp = model_afolu(df_peru_tmp, )

In [ ]:
flds = [x for x in df_afolu.columns if ("lvst" in x) or ("lsmm" in x)]
df_afolu[flds].iloc[0]

In [ ]:
modvar = matt.get_variable("Livestock Head Count")

df_tmp[[x for x in df_tmp.columns if x.startswith("emission_co2e_subsector_total") and (("lsmm" in x) or ("lvst" in x))]]

In [ ]:
# matt.get_attribute_table("Agriculture").key_values

In [ ]:
# [x for x in df_afolu.columns if x not in matt.all_variable_fields_input]